In [1]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import numpy as np
import pyphi
import visualize_pyphi
from pyphi.models.subsystem import FlatCauseEffectStructure
import visualize_pyphi.utils as utils
import visualize_pyphi.network_generator as network_generator
from tqdm.auto import tqdm
import glob
import time
from datetime import timedelta
import pandas as pd

pyphi.config.CES_DISTANCE='ID'

# Make network

In [2]:
# Chapter 3 network

net_name = "ch3"
n=6

# system parameters
strong = 1
medium = 0.2
weak = 0.04
none = 0

# input output
ff = 0.05
fb = 0.005

# naka rushdon params
th = 1 / 4
exp = 5

weights = np.array(
    [
        [strong, none, weak, medium, 0, 0],
        [medium, weak, none, strong, fb, 0],
        [weak, strong, medium, none, 0, 0],
        [none, medium, strong, weak, 0, ff],
        [0, ff, 0, 0, 0, 0],
        [0, 0, 0, fb, 0, 1],
    ]
)
weights[:4,:4] /= weights[:4,:4].sum(1)

node_labels = ["A", "B", "C", "D", "I", "O"]
mech_func = ["s"] * n

network = visualize_pyphi.network_generator.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=False,
)

  0%|          | 0/64 [00:00<?, ?it/s]

# Plot weights and tpms

In [29]:
wdf = pd.DataFrame(np.round(weights,3),index=node_labels,columns=node_labels)
wdf

,A,B,C,D,I,O
A,0.806,0.000,0.032,0.161,0.0,0.0
B,0.161,0.032,0.000,0.806,0.0,0.0
C,0.032,0.806,0.161,0.000,0.0,0.0
D,0.000,0.161,0.806,0.032,0.0,1.0
I,0.000,0.050,0.000,0.000,0.0,0.0
O,0.000,0.000,0.000,0.005,0.0,0.0


In [30]:
sbs = np.log(pyphi.convert.state_by_node2state_by_state(network.tpm)+0.01)
states = [utils.strp(str(s)) for s in list(pyphi.utils.all_states(network.size))]

import plotly.express as px

fig = px.imshow(sbs,x=states,y=states,color_continuous_scale='Greys')
fig.update_xaxes(side="top")
fig.update_layout(height=1200,width=2000,xaxis={'visible': False, 'showticklabels': False},yaxis={'visible': False, 'showticklabels': False},coloraxis_showscale=False)
fig.show()
fig.write_image(f'figures/ch3/{net_name}_tpm_nolabels.png')



fig = px.imshow(sbs,x=states,y=states,color_continuous_scale='Greys')
fig.update_xaxes(side="top")
fig.update_layout(height=1200,width=2000,coloraxis_showscale=False)
fig.show()
fig.write_image(f'figures/ch3/{net_name}_tpm_labels.png')

# standard system

In [3]:
state = (0, 1, 0, 0, 1, 0)
subsystem = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3))
ces = pyphi.compute.ces(subsystem)
sepces = FlatCauseEffectStructure(ces)

In [4]:
relations = list(pyphi.relations.relations(subsystem, sepces, max_order=2, parallel=True, parallel_kwargs=dict(n_jobs=200,batch_size=10000,verbose=5)))
# relations = utils.loadpkl('ch3_standard_ces_rels.pkl')

[Parallel(n_jobs=200)]: Using backend LokyBackend with 200 concurrent workers.
[Parallel(n_jobs=200)]: Done  53 out of 322 | elapsed:    5.4s remaining:   27.5s
[Parallel(n_jobs=200)]: Done 118 out of 322 | elapsed:    9.6s remaining:   16.6s
[Parallel(n_jobs=200)]: Done 183 out of 322 | elapsed:   13.7s remaining:   10.4s
[Parallel(n_jobs=200)]: Done 248 out of 322 | elapsed:   17.3s remaining:    5.1s
[Parallel(n_jobs=200)]: Done 313 out of 322 | elapsed:   20.9s remaining:    0.6s
[Parallel(n_jobs=200)]: Done 322 out of 322 | elapsed:   21.4s finished


In [40]:
pyphi_ces = pyphi.big_phi.sia(
    subsystem,
    ces,
    relations,
    parallel=True,
    check_trivial_reducibility=True,
)

In [41]:
pyphi_ces.phi

0.004751469099476694

In [42]:
utils.sepces2df(FlatCauseEffectStructure(pyphi_ces.ces),subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.469923
1,A,EFFECT,A,0,0.613822
2,D,CAUSE,B,0,0.469923
3,D,EFFECT,C,0,0.613822
4,AB,CAUSE,ABC,011,0.020929
5,AB,EFFECT,D,0,0.152428
6,AC,CAUSE,AD,01,0.014019
7,AC,EFFECT,AC,00,0.017256
8,AD,CAUSE,AB,00,0.060151
9,AD,EFFECT,CD,00,0.017256


In [5]:
bjces = visualize_pyphi.compute.filter_ces(subsystem, sepces, None, max_relations_k=3, parallel=True)

  0%|          | 0/432 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend LokyBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done  84 tasks      | elapsed:    4.4s
[Parallel(n_jobs=120)]: Done 288 tasks      | elapsed:   13.0s
[Parallel(n_jobs=120)]: Done 367 out of 432 | elapsed:   17.9s remaining:    3.2s
[Parallel(n_jobs=120)]: Done 432 out of 432 | elapsed:   21.4s finished


In [6]:
bjces[0]['big phi']

88.92752545027056

In [8]:
utils.sepces2df(bjces[0]['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.469923
1,A,EFFECT,A,0,0.613822
2,C,CAUSE,D,0,0.469923
3,C,EFFECT,B,0,0.673202
4,D,CAUSE,B,0,0.469923
5,D,EFFECT,C,0,0.613822
6,AB,CAUSE,ABC,011,0.020929
7,AB,EFFECT,D,0,0.152428
8,AC,CAUSE,AD,01,0.014019
9,AC,EFFECT,AC,00,0.017256


In [32]:
# allrels = []
# for k in tqdm(range(2,len(ces)+1)):
#     print(f'Computing k = {k}...')
#     krels = list(pyphi.relations.relations(subsystem_standard,ces,min_order=k,max_order=k,parallel=True,parallel_kwargs=dict(n_jobs=120,batch_size=10000,verbose=5)))
#     allrels.append(krels)

# utils.pklthis(allrels,'ch3_allrels.pkl')

# allrels = utils.loadpkl('ch3_allrels.pkl')

# allrelsflat = utils.flatten(allrels)

# standard_ces_allrels = compute.filter_ces(subsystem_standard, ces, None, relations=allrelsflat, parallel=True, n_jobs=200)

# pklthis(standard_ces_allrels,f'{net_name}_ces_allrels.pkl')

# Plotting

In [34]:
# plotting FULL CES
fig = viz.plot_ces(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    network_name="figures/ch3/standard_all",
    show_legend=False,
)

Computing edges:   0%|          | 0/73 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/136 [00:00<?, ?it/s]

In [36]:
# plotting MESH
fig = viz.plot_ces(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    network_name="figures/ch3/standard_MESH",
    png_resolution=(3000,2000),    
    show_legend=False,    
    show_edges=False,
    show_mesh=True,
    show_links=False,
    show_labels=False,
    show_mechanism_labels=False,
    show_purview_labels=False,
    show_mechanism_base=False,
    show_chains=False,    
)

Computing triangles:   0%|          | 0/136 [00:00<?, ?it/s]

In [37]:
# plotting EDGES
fig = viz.plot_ces(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    network_name="figures/ch3/standard_EDGES",
    show_legend=False,    
    png_resolution=(3000,2000),    
    show_edges=True,
    show_mesh=False,
    show_links=False,
    show_labels=False,
    show_mechanism_base=False,
    show_chains=False,    
)

Computing edges:   0%|          | 0/73 [00:00<?, ?it/s]

In [38]:
# plotting LABELS
fig = viz.plot_ces(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    network_name="figures/ch3/standard_LABELS",
    show_legend=False,    
    png_resolution=(3000,2000),    
    show_edges=False,
    show_mesh=False,
    show_links=False,
    show_labels=True,
    show_mechanism_base=False,
    show_chains=False,    
)

In [39]:
# plotting BASE
fig = viz.plot_ces(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    network_name="figures/ch3/standard_BASE",
    show_legend=False,    
    png_resolution=(3000,2000),    
    show_edges=True,
    transparent_edges=True,
    show_mesh=False,
    show_links=False,
    show_labels=False,
    show_mechanism_base=True,
    show_chains=True,
    
)

Computing edges:   0%|          | 0/73 [00:00<?, ?it/s]

In [40]:
# plotting LINKS
fig = viz.plot_ces(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    network_name="figures/ch3/standard_LINKS",
    show_legend=False,    
    png_resolution=(3000,2000),    
    show_edges=True,
    transparent_edges=True,
    show_mesh=False,
    show_links=True,
    show_labels=False,
    show_mechanism_base=False,
    show_chains=False,
    
)

Computing edges:   0%|          | 0/73 [00:00<?, ?it/s]

# Information

## Consequence

In [41]:
state = (0, 0, 0, 1, 0, 1)
subsystem_consequence = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(subsystem_consequence, parallel=True)]
)

consequence_ces = compute.filter_ces(
    subsystem_consequence, ces, None, max_relations_k=3, parallel=True
)

  0%|          | 0/288 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=120)]: Done  25 tasks      | elapsed:    1.0s
[Parallel(n_jobs=120)]: Done  48 tasks      | elapsed:    1.9s
[Parallel(n_jobs=120)]: Done  78 out of 288 | elapsed:    2.5s remaining:    6.6s
[Parallel(n_jobs=120)]: Done 107 out of 288 | elapsed:    3.5s remaining:    6.0s
[Parallel(n_jobs=120)]: Done 136 out of 288 | elapsed:    4.4s remaining:    4.9s
[Parallel(n_jobs=120)]: Done 165 out of 288 | elapsed:    5.0s remaining:    3.7s
[Parallel(n_jobs=120)]: Done 194 out of 288 | elapsed:    5.4s remaining:    2.6s
[Parallel(n_jobs=120)]: Done 223 out of 288 | elapsed:    6.3s remaining:    1.8s
[Parallel(n_jobs=120)]: Done 252 out of 288 | elapsed:    6.8s remaining:    1.0s
[Parallel(n_jobs=120)]: Done 281 out of 288 | elapsed:    7.3s remaining:    0.2s
[Parallel(n_jobs=120)]: Done 288 out of 288 | elapsed:    7.8s fi

In [42]:
# plotting
fig = viz.plot_ces(
    subsystem_consequence,
    consequence_ces[0]["ces"],
    consequence_ces[0]["relations"],
    network_name="figures/ch3/information_1",
    show_legend=False,    
    png_resolution=(3000,2000),       
)

Computing edges:   0%|          | 0/76 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/158 [00:00<?, ?it/s]

## All off

In [43]:
state = (0, 0, 0, 0, 0, 0)
subsystem_all_off = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(subsystem_all_off, parallel=True)]
)

all_off_ces = compute.filter_ces(
    subsystem_all_off, ces, None, max_relations_k=3, parallel=True
)

  0%|          | 0/64 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done   7 out of  64 | elapsed:    0.9s remaining:    7.1s
[Parallel(n_jobs=120)]: Done  14 out of  64 | elapsed:    0.9s remaining:    3.4s
[Parallel(n_jobs=120)]: Done  21 out of  64 | elapsed:    1.0s remaining:    2.1s
[Parallel(n_jobs=120)]: Done  28 out of  64 | elapsed:    1.1s remaining:    1.5s
[Parallel(n_jobs=120)]: Done  35 out of  64 | elapsed:    1.2s remaining:    1.0s
[Parallel(n_jobs=120)]: Done  42 out of  64 | elapsed:    1.6s remaining:    0.8s
[Parallel(n_jobs=120)]: Done  49 out of  64 | elapsed:    1.7s remaining:    0.5s
[Parallel(n_jobs=120)]: Done  56 out of  64 | elapsed:    1.9s remaining:    0.3s
[Parallel(n_jobs=120)]: Done  64 out of  64 | elapsed:    2.1s finished


In [44]:
# plotting
fig = viz.plot_ces(
    subsystem_all_off,
    all_off_ces[0]["ces"],
    all_off_ces[0]["relations"],
    network_name="figures/ch3/information_2",
    show_legend=False,    
    png_resolution=(3000,2000),           
)

Computing edges:   0%|          | 0/89 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/180 [00:00<?, ?it/s]

# Integration

In [45]:
plots.plot_effect_of_MIP(
    subsystem_standard,
    standard_ces[0]['ces'],
    standard_ces[0]['relations'],
    "figures/ch3/integration",
    partitions=None,
    common_kwargs=dict(show_legend=False,    
    png_resolution=(3000,2000))
)

Computing triangles:   0%|          | 0/136 [00:00<?, ?it/s]

Not redrawing base


Computing edges:   0%|          | 0/22 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/53 [00:00<?, ?it/s]

Not redrawing base


# Exclusion

## Smaller system

In [ ]:

state = (0, 1, 0, 0, 1, 0)
smaller_subsystem = pyphi.Subsystem(network, state, nodes=(0, 1, 2))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(smaller_subsystem, parallel=True)]
)

smaller_ces = compute.filter_ces(smaller_subsystem, ces, None, max_relations_k=3, parallel=True)



Computing concepts:  14%|██████████████                                                                                    | 1/7 [01:10<07:02, 70.36s/it]
                                                                                                                                                         

In [ ]:
# plotting
fig = viz.plot_ces(
    smaller_subsystem,
    smaller_ces[0]["ces"],
    smaller_ces[0]["relations"],
    network_name="figures/ch3/exclusion_1",
    show_legend=False,    
    png_resolution=(3000,2000),
    floor_width_scales=[2, 2, 2],    
    floor_height_scale=[1,1,1],
    base_height_scale=1,
    base_z_offset=0.3,
    base_width_scale=1,
    cause_effect_distance=0.3,
#     eye_coordinates=(-0.3, -0.2, 0.025),
    
)

## Larger system

In [117]:
state = (0, 1, 0, 0, 1, 0)
larger_subsystem = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3, 5))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(larger_subsystem, parallel=True)]
)

larger_ces = compute.filter_ces(larger_subsystem, ces, None, max_relations_k=3, parallel=True, n_jobs=200)

  0%|          | 0/104976 [00:00<?, ?it/s]

[Parallel(n_jobs=200)]: Using backend LokyBackend with 200 concurrent workers.
[Parallel(n_jobs=200)]: Done 176 tasks      | elapsed:   13.9s
[Parallel(n_jobs=200)]: Done 3000 tasks      | elapsed:  4.8min
[Parallel(n_jobs=200)]: Done 13296 tasks      | elapsed: 11.9min
[Parallel(n_jobs=200)]: Done 25464 tasks      | elapsed: 22.2min
/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=200)]: Done 39504 tasks      | elapsed: 36.9min
[Parallel(n_jobs=200)]: Done 55416 tasks      | elapsed: 54.2min
[Parallel(n_jobs=200)]: Done 73200 tasks      | elapsed: 76.0min
[Parallel(n_jobs=200)]: Done 92856 tasks      | elapsed: 93.2min
[Parallel(n_jobs=200)]: Done 104976 out of 104976 | elapsed: 107.4min finished


In [119]:
larger_ces[0]['big phi']

786.2557699484865

In [120]:
larger_ces[0]['MIP']

(((1,), (0, 2, 3, 5)), (0, 2, 3, 5), (1,), EFFECT)

In [121]:
utils.sepces2dflarger_ces[0]['ces']

════════════════════════════════
Cause-effect structure (32 concepts)
════════════════════════════════
  Maximally-irreducible cause  
    φ = 0.469923  
    Mechanism: [A]  
    Purview: [A]  
    State: [0]  
    Direction: CAUSE  
  Maximally-irreducible effect  
    φ = 0.613822  
    Mechanism: [A]  
    Purview: [A]  
    State: [0]  
    Direction: EFFECT  
  Maximally-irreducible cause  
    φ = 0.469923  
    Mechanism: [C]  
    Purview: [D]  
    State: [0]  
    Direction: CAUSE  
  Maximally-irreducible effect  
    φ = 0.673202  
    Mechanism: [C]  
    Purview: [B]  
    State: [1]  
    Direction: EFFECT  
  Maximally-irreducible cause  
    φ = 0.473377  
    Mechanism: [D]  
    Purview: [B]  
    State: [1]  
    Direction: CAUSE  
  Maximally-irreducible effect  
    φ = 0.613822  
    Mechanism: [D]  
    Purview: [C]  
    State: [0]  
    Direction: EFFECT  
  Maximally-irreducible cause  
    φ = 0.642097  
    Mechanism: [O]  
    Purview: [O]  
    State: [0]

In [11]:
# larger_relations = list(
#     pyphi.relations.relations(
#         larger_subsystem,
#         ces,
#         min_order=2,
#         max_order=3,
#         parallel=True,
#         parallel_kwargs=dict(n_jobs=200, batch_size=10000, verbose=2),
#     )
# )

# utils.pklthis(larger_relations,'chr_larger_2-3rels.pkl')

In [9]:
larger_relations = utils.loadpkl('chr_larger_2-3rels.pkl')

In [15]:
utils.pklthis(larger_ces[0]['MIP'],'larger_ces_MIP.pkl')

In [16]:
utils.pklthis(larger_ces[0]['ces'],'larger_ces_CES.pkl')

In [22]:
larger_ces[0]['big phi']

0.0

In [21]:
larger_ces[0]['MIP']

(((0, 1, 2, 3), (5,)), (0, 1, 2, 3), (5,), CAUSE)

In [20]:
utils.sepces2df(larger_ces[0]['ces'],larger_subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.469923
1,A,EFFECT,A,0,0.613822
2,B,CAUSE,C,1,0.995828
3,B,EFFECT,D,1,0.695679
4,C,CAUSE,D,0,0.469923
5,C,EFFECT,B,0,0.673202
6,D,CAUSE,B,0,0.473377
7,D,EFFECT,CO,00,0.642478
8,AC,CAUSE,AD,01,0.014019
9,AC,EFFECT,AC,00,0.017256


In [ ]:
# plotting
fig = viz.plot_ces(
    larger_subsystem,
    larger_ces[0]["ces"],
    larger_ces[0]["relations"],
    network_name="figures/ch3/exclusion_2",
    show_legend=False,    
    png_resolution=(3000,2000),
    floor_width_scales=[1.25 for x in range(len(list(pyphi.utils.powerset(larger_subsystem.node_indices))))],
    floor_height_scale=[1.75 for x in range(len(list(pyphi.utils.powerset(larger_subsystem.node_indices))))],
    base_height_scale=5,
    base_z_offset=-.2,
     base_width_scale=.6,
#     cause_effect_distance=0.6,    
    hovermode=False,    
    eye_coordinates=(0.2, -0.4, 0.025),
)